# Exercise 01:  Minimal high-level UQ example

This notebook demonstrates high-level UQ with CUQIpy on a 1D Deconvolution test problem.

## Learning objectives of this notebook:
- Get acquainted with the CUQIpy components needed to specify a Bayesian inverse problem.
- Run a high-level UQ analysis of an inverse problem.

## Table of contents: 
* [1. UQ in five lines!](#UQ5)
* [2. Loading a test problem](#TestProblem)
* [3. Specifying and solving a Bayesian inverse problem ](#Bayesian)
* [4. ★ Changing the prior](#ChangingPrior)
* [5. ★ Gibbs sampling for hierarchical problems](#Gibbs)

*The last two sections marked by "★" are optional - try running through sections 1, 2, 3 before attempting the exercises, and before sections 4 and 5.*

First we import any python packages we need, here simply NumPy to handle array computations:

In [ ]:
import numpy as np

Then we import the functionality we need from CUQIpy:

In [ ]:
import sys
sys.path.append("../../CUQIpy/")

from cuqi.distribution import Gaussian, Laplace_diff, JointDistribution, Gamma, GaussianCov, Cauchy_diff
from cuqi.problem import BayesianProblem
from cuqi.testproblem import Deconvolution1D

## 1. UQ in five lines!   <a class="anchor" id="UQ5"></a> 

In just five lines of CUQIpy code we can:
- load a forward model and data from a 1D deconvolution test problem,
- specify a prior,
- specify a distribution for the data,
- formulate a Bayesian inverse problem with observed data, and
- run a UQ analysis:

In [ ]:
model, data, probInfo = Deconvolution1D.get_components(dim=50, phantom="Square")

In [ ]:
x = Gaussian(mean=np.zeros(50), std=0.2)

In [ ]:
y = Gaussian(mean=model@x, std=0.05)

In [ ]:
BP = BayesianProblem(y,x).set_data(y=data)

In [ ]:
BP.UQ(exact=probInfo.exactSolution)

In the following sections of this notebook we break it down and take a slightly closer look at each step. The subsequent notebooks provide additional details.

## 2. Loading a test problem  <a class="anchor" id="TestProblem"></a> 

We specify a small 1D deconvolution test problem:

In [ ]:
model, data, probInfo = Deconvolution1D.get_components(dim=50, phantom="Square")

Note that the test problem can be configured (e.g. other phantoms, noise types and level etc.) by means of the inputs, see `help(Deconvolution1D)` for details.

Several other test problems are available:
- `Deconvolution1D`: 1D periodic deconvolution problem.
- `Deconvolution1D`: 2D deblurring problem.
- `Heat_1D`:  Heat equation PDE problem.
- `Poisson_1D`: Poisson equation PDE problem.
- `Abel_1D`:  Abel equation PDE (1D rotationally symmetric CT) problem. 

These are imported as follows:

In [ ]:
from cuqi.testproblem import Deconvolution2D, Heat_1D, Poisson_1D, Abel_1D

and their calling signature is the same as for the `Deconvolution1D` problem, e.g. 
```
model, data, probInfo = Heat_1D.get_components()
```
Input arguments vary and default values are provided if left empty. Calling help of each testproblem, e.g., `help(Heat_1D)` will describe the test problem and the inputs it accepts.

For now, proceeding with the `Deconvolution1D` test problem, we take a look at the model and see that it is a CUQIpy LinearModel:

In [ ]:
model

We take a look at the data:

In [ ]:
data

The data is a `CUQIarray`, which is a normal NumPy array further equipped with a few utilities, such as Geometry, which allows us to do plotting conveniently:

In [ ]:
data.plot()

The last thing returned by the test problem was `probInfo` which contains additional information about the test problem, typically it includes the exact solution (phantom) and the exact data. We take a look at both:

In [ ]:
probInfo

In [ ]:
probInfo.exactSolution.plot()

In [ ]:
probInfo.exactData.plot()

Since `CUQIarray` is a NumPy array (technically subclassed from NumPy ndarray), we can do all computations that NumPy admits and still get a `CUQIarray`, for example take the difference between the data and exact data and call the plot method:

In [ ]:
(data-probInfo.exactData).plot()

## 3. Specifying and solving a Bayesian inverse problem  <a class="anchor" id="Bayesian"></a> 

The deconvolution test problem is a linear problem with additive noise:

$$ \mathbf{y} = \mathbf{A}\mathbf{x}+\mathbf{e},$$
where $\mathbf{A}\in\mathbb{R}^{n\times n}$, $\mathbf{x}, \mathbf{b}\in\mathbb{R}^n$ 
and the elements of $\mathbf{x}$ are i.i.d. zero-mean Gaussian with standard deviation $\sigma_x$ and the additive noise is also i.i.d. Gaussian with zero mean and standard deviation $\sigma_e$:
$$
\mathbf{x}\sim\mathcal{N}(\mathbf{0},\sigma_x^2\mathbf{I}_n), \\
\mathbf{e}\sim\mathcal{N}(\mathbf{0},\sigma_e^2\mathbf{I}_n).$$

An equivalent, more compact way, to specify this problem is
$$
\mathbf{x}\sim\mathcal{N}(\mathbf{0},\sigma_x^2\mathbf{I}_n), \\
\mathbf{y}\sim\mathcal{N}(\mathbf{Ax},\sigma_e^2\mathbf{I}_n).$$

We can specify these distributions in CUQIpy as follows. For $\mathbf{x}$ with a particular fixed choice of $\mathbf{\sigma_x}$:

In [ ]:
x = Gaussian(mean=np.zeros(50), std=0.2)

We create the Gaussian distribution with the model applied to `x` as mean and a choice of fixed standard deviation:

In [ ]:
y = Gaussian(model@x, std=0.05)

The prior, likelihood are combined in a Bayesian inverse problem:

In [ ]:
BP = BayesianProblem(y, x)
print(BP)

This is essentially a joint distribution over `x` and `y`. The last step is to provide the observed data for `y`:

In [ ]:
BP.set_data(y=data)

The "completely non-expert approach" to solving (more detailed approaches described in later notebooks) is to simply run the UQ method:

In [ ]:
BP.UQ()

The `UQ` method looks at the components of the inverse problem, chooses a suitable sampler, samples the posterior and presents results visually.

To compare with the exact solution (if available) one can pass it as an input, and we can also return the posterior samples:

In [ ]:
samples = BP.UQ(exact=probInfo.exactSolution)

Having returned the samples allows us to further investigate the posterior in addition to the prefined analysis done by `.UQ()`, for example we can plot 5 randomly chosen samples:

In [ ]:
samples.plot()

## Try yourself (optional):  
- Play with the parameters of the Gaussian prior to see if a better solution can be obtained.
- Change phantom to another of the options provided by the TestProblem. Hint: use `help(Deconvolution1D)` to see which phantoms are available.
- Try changing to the `Deconvolution2D` test problem and run the same code.

## 4. ★ Changing the prior <a class="anchor" id="ChangingPrior"></a> 

It is straightforward to change components of the BayesianProblem. For example if we want to experiment with a different prior we can easily swap it out.

We specify a `Laplace_diff` prior, which is a Laplace distribution on differences between neighbouring function values:

In [ ]:
x = Laplace_diff(location=np.zeros(50), scale=0.01)

We create a new Bayesian Problem:

In [ ]:
BP_lap = BayesianProblem(y, x)
print(BP_lap)

Set the data:

In [ ]:
BP_lap.set_data(y=data)

And rerun the `UQ` method:

In [ ]:
samples_lap = BP_lap.UQ(exact=probInfo.exactSolution)

Note how a different sampler was chosen due to the change of prior, and how the prior has changed the solution to be more similar to the exact solution.

We also plot a few selected samples and note the more piece-wise constant appearance:

In [ ]:
samples_lap.plot()

## Try yourself (optional):  
- Play with the parameters of the Laplace prior to see if a better solution can be obtained.
- Try the `Cauchy_diff` prior instead with scale parameter 0.01 (only for `Deconvolution1D`) for an even more edge-preserving prior, which however is more difficult to sample reliably.

# 5. ★ Gibbs sampling for hierarchical problems   <a class="anchor" id="Gibbs"></a> 

Until now we assumed that the mean, std and scale parameters of the prior and noise distributions were known. If these parameters are unknown, we include them as parameters to be estimated in a joint Bayesian inverse problem. Only very few additional lines of CUQIpy are needed to achieve this, through a hierarchical prior.

For example we can assume weakly informative Gamma distribution prior distributions on $\sigma_x$ and $\sigma_e$ such that we have the four random variables:
$$
\sigma_x\sim \mathrm{Gamma}(1, 1\cdot 10^{-2}), \\
\sigma_e\sim \mathrm{Gamma}(1, 1\cdot 10^{-2}), \\
\mathbf{x}\sim\mathcal{N}(\mathbf{0},\sigma_x^2\mathbf{I}_n), \\
\mathbf{y}\sim\mathcal{N}(\mathbf{Ax},\sigma_e^2\mathbf{I}_n).$$

We specify the four distributions in CUQIpy. To specify the hierarchical structure we use python lambda functions (equivalent to anonymous functions in MATLAB):

In [ ]:
sig_x = Gamma(1, 1e-2)
sig_e = Gamma(1, 1e-2)
x = GaussianCov(np.zeros(50), cov=lambda sig_x : 1/sig_x)
y = GaussianCov(model@x, cov=lambda sig_e : 1/sig_e)

and proceed as before to specify the Bayesian Problem, now with the four distributions:

In [ ]:
BP = BayesianProblem(y, x, sig_x, sig_e)
print(BP)

Again, we provide the observed data for `y`:

In [ ]:
BP.set_data(y=data)

As before, we can run the UQ method, which will use a Gibbs sampler automatically selecting suitable sampling methods for each of the three parameters to be sampled `x`, `sig_x` and `sig_e`. We provide the true solution for `x` and the true squared std for the noise for comparison in the generated plots:

In [ ]:
samplesH = BP.UQ(Ns=2000, exact={"x":probInfo.exactSolution, "sig_e":400})

Earlier we saw that the Laplace_diff prior gave a more piecewise constant behavior of `x`and we can also try employing that as part of the hierarchical problem:

In [ ]:
x  = Laplace_diff(np.zeros(50), lambda sig_x: 1/sig_x)

We set up the Bayesian Problem and specify the observed data for `y`:

In [ ]:
BP = BayesianProblem(y, x, sig_x, sig_e)
BP.set_data(y=data)

And we sample it using the `UQ` method, this time returning the generated samples:

In [ ]:
samplesH = BP.UQ(Ns=2000, exact={"x":probInfo.exactSolution, "sig_e":400})

We see the effect of the Laplace_diff prior once again on the solution, and in addition we see trace plots for the `sig_x` and `sig_e` parameters produced by the `UQ` method with the Gibbs sampler.

Having returned the samples we can pick out the individual parameters and do separate plots manually:

In [ ]:
samplesH["x"].plot()

In [ ]:
samplesH["sig_x"].plot_trace()